In [1]:
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ==================== RLE Encoding/Decoding ====================

def rle_encode_instance_mask(mask: np.ndarray) -> str:
    """Convert instance segmentation mask (H,W) -> RLE triple string."""
    pixels = mask.flatten(order="F").astype(np.int32)
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    
    rle = []
    for i in range(len(runs)-1):
        start = runs[i]
        end = runs[i+1]
        length = end - start
        val = pixels[start]
        if val > 0:
            rle.extend([val, start, length])
    
    return "0" if not rle else " ".join(map(str, rle))


def rle_decode_instance_mask(rle: str, shape: tuple) -> np.ndarray:
    """Convert RLE triple string back into instance mask."""
    if not rle or str(rle).strip() in ("", "0", "nan"):
        return np.zeros(shape, dtype=np.uint16)
    
    s = list(map(int, rle.split()))
    mask = np.zeros(shape[0] * shape[1], dtype=np.uint16)
    
    for i in range(0, len(s), 3):
        val, start, length = s[i], s[i+1], s[i+2]
        mask[start-1:start-1+length] = val
    
    return mask.reshape(shape, order="F")


def rle_to_semantic_mask(rle: str, shape: tuple, class_id: int) -> np.ndarray:
    """Convert RLE to semantic segmentation mask (0=background, class_id=foreground)."""
    instance_mask = rle_decode_instance_mask(rle, shape)
    semantic_mask = (instance_mask > 0).astype(np.uint8) * class_id
    return semantic_mask


# ==================== Dataset ====================

# ==================== Dataset (Modified) ====================

class NucleiDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, is_train=True, target_class=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.transform = transform
        self.is_train = is_train
        # target_class is the name of the cell type to segment (or None for single-model training)
        self.target_class = target_class 
        self.classes = ['Epithelial', 'Lymphocyte', 'Neutrophil', 'Macrophage']
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_id = row['image_id']
        
        # Load image
        # ... (image loading logic remains the same) ...
        img_path = self.img_dir / f"{img_id}.tif"
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w = image.shape[:2]
        
        if self.is_train:
            if self.target_class:
                # Binary Training: Create a mask for only the target class (1) vs background (0)
                mask = np.zeros((h, w), dtype=np.uint8)
                rle = row[self.target_class]
                # Use class_id=1 for the foreground class
                class_mask = rle_to_semantic_mask(rle, (h, w), 1) 
                mask = np.maximum(mask, class_mask)
            else:
                # Original Multi-class Training (fallback if needed)
                mask = np.zeros((h, w), dtype=np.uint8)
                for class_idx, class_name in enumerate(self.classes, start=1):
                    rle = row[class_name]
                    class_mask = rle_to_semantic_mask(rle, (h, w), class_idx)
                    mask = np.maximum(mask, class_mask)
            
            # Apply augmentations
            # ... (transform logic remains the same) ...
            if self.transform:
                transformed = self.transform(image=image, mask=mask)
                image = transformed['image']
                mask = transformed['mask']
            
            return image, mask.long()
        else:
            # ... (Inference logic remains the same) ...
            if self.transform:
                transformed = self.transform(image=image)
                image = transformed['image']
            return image, img_id



# ==================== Model (Modified for Binary) ====================

def get_model(num_classes=2, encoder='resnet50', encoder_weights='imagenet'):
    """Create segmentation model. num_classes=2 (background + foreground class)"""
    model = smp.UnetPlusPlus(
        encoder_name=encoder,
        encoder_weights=encoder_weights,
        in_channels=3,
        classes=num_classes, # Should be 2 for binary
        activation=None # Use None so we can apply softmax/sigmoid and then argmax
    )
    return model


# ==================== Loss Functions ====================

class FocalLoss(nn.Module):
    """Focal Loss for handling class imbalance"""
    def __init__(self, alpha=None, gamma=2, ignore_index=-100):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
    
    def forward(self, inputs, targets):
        ce_loss = nn.functional.cross_entropy(
            inputs, targets, reduction='none', ignore_index=self.ignore_index
        )
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        
        if self.alpha is not None:
            alpha_t = self.alpha[targets]
            focal_loss = alpha_t * focal_loss
        
        return focal_loss.mean()


class CombinedLoss(nn.Module):
    """Combination of Focal Loss and Dice Loss"""
    def __init__(self, num_classes=5, device='cpu'):
        super().__init__()
        # Class weights: [background, Epithelial, Lymphocyte, Neutrophil, Macrophage]
        # Higher weights for rare classes
        self.alpha = torch.tensor([0.5, 1.0, 2.0, 10.0, 10.0])
        self.device = device
        self.focal = FocalLoss(alpha=None, gamma=2)  # We'll handle alpha manually
        self.dice = smp.losses.DiceLoss(mode='multiclass')
    
    def forward(self, pred, target):
        # Move alpha to the same device as target if needed
        if self.alpha.device != target.device:
            self.alpha = self.alpha.to(target.device)
        
        # Compute focal loss with device-aware alpha
        ce_loss = nn.functional.cross_entropy(pred, target, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** 2) * ce_loss
        
        # Apply class weights
        alpha_t = self.alpha[target]
        focal_loss = (alpha_t * focal_loss).mean()
        
        # Compute dice loss
        dice_loss = self.dice(pred, target)
        
        return focal_loss + dice_loss

# ==================== Loss Functions (Modified for Binary) ====================

class CombinedBinaryLoss(nn.Module):
    """Combination of Focal Loss and Dice Loss for binary segmentation"""
    def __init__(self, device='cpu'):
        super().__init__()
        # Binary Class weights: [background (0), foreground (1)]
        # You'll likely need to tune this! Example: more weight for the foreground
        self.alpha = torch.tensor([0.5, 5.0]) 
        self.device = device
        # Use 'binary' mode for DiceLoss
        self.dice = smp.losses.DiceLoss(mode='binary') 
    
    def forward(self, pred, target):
        # Move alpha to the same device
        if self.alpha.device != target.device:
            self.alpha = self.alpha.to(target.device)
        
        # Focal Loss logic for binary cross-entropy (similar to multiclass, but targets are 0/1)
        ce_loss = nn.functional.cross_entropy(pred, target, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** 2) * ce_loss
        
        # Apply class weights
        alpha_t = self.alpha[target]
        focal_loss = (alpha_t * focal_loss).mean()
        
        # Dice loss expects predictions to be class 1 probability (after softmax/sigmoid)
        # and targets as float. For binary, we use the probability of class 1.
        dice_loss = self.dice(torch.softmax(pred, dim=1)[:, 1], target.float()) 
        
        return focal_loss + dice_loss

# ==================== Training ====================

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    pbar = tqdm(loader, desc='Training')
    for images, masks in pbar:
        images = images.to(device)
        masks = masks.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    return total_loss / len(loader)


def validate_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for images, masks in tqdm(loader, desc='Validation'):
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss.item()
    
    return total_loss / len(loader)


# ==================== Post-processing for Instance Segmentation ====================

def semantic_to_instance(semantic_mask, min_size=10):
    """Convert semantic mask to instance mask using connected components"""
    instance_mask = np.zeros_like(semantic_mask, dtype=np.uint16)
    instance_id = 1
    
    # Apply connected components for each predicted nucleus
    num_labels, labels = cv2.connectedComponents(semantic_mask.astype(np.uint8))
    
    for i in range(1, num_labels):
        component = (labels == i)
        if component.sum() >= min_size:
            instance_mask[component] = instance_id
            instance_id += 1
    
    return instance_mask

# ==================== Post-processing (Modified for Binary Prediction) ====================

def predict_binary_mask(model, image, device, threshold=0.5):
    """Predict a single binary mask from a binary segmentation model."""
    model.eval()
    
    with torch.no_grad():
        image_tensor = image.unsqueeze(0).to(device)
        output = model(image_tensor) # Output shape (1, 2, H, W)
        
        # Apply softmax to get probabilities for class 0 and 1
        probabilities = torch.softmax(output, dim=1)[:, 1] # Get probabilities for class 1
        
        # Apply threshold to create a binary mask (0 or 1)
        binary_mask = (probabilities > threshold).cpu().numpy()[0].astype(np.uint8)
    
    return binary_mask


def predict_instances(model_dict, image, device, min_size=10, threshold=0.5):
    """Predict instance masks by iterating over multiple models."""
    instance_masks = {}
    
    for class_name, model in model_dict.items():
        # Get the binary semantic mask for this class
        semantic_mask_binary = predict_binary_mask(model, image, device, threshold)
        
        # Convert the binary semantic mask to an instance mask
        instance_mask = semantic_to_instance(semantic_mask_binary, min_size=min_size)
        instance_masks[class_name] = instance_mask
    
    return instance_masks

# def predict_instances(model, image, device, min_size=10):
#     """Predict instance masks for all 4 classes"""
#     model.eval()
    
#     with torch.no_grad():
#         image_tensor = image.unsqueeze(0).to(device)
#         output = model(image_tensor)
#         pred = torch.softmax(output, dim=1)
#         pred = torch.argmax(pred, dim=1).cpu().numpy()[0]
    
#     # Separate predictions by class
#     instance_masks = {}
#     classes = ['Epithelial', 'Lymphocyte', 'Neutrophil', 'Macrophage']
    
#     for class_idx, class_name in enumerate(classes, start=1):
#         class_mask = (pred == class_idx).astype(np.uint8)
#         instance_mask = semantic_to_instance(class_mask, min_size=min_size)
#         instance_masks[class_name] = instance_mask
    
#     return instance_masks


# ==================== Main Pipeline ====================

# def main():
#     # Configuration
#     DATA_DIR = Path('kaggle-data')
#     TRAIN_DIR = DATA_DIR / 'train'
#     TEST_DIR = DATA_DIR / 'test_final'
#     TRAIN_CSV = DATA_DIR / 'train_ground_truth.csv'
    
#     BATCH_SIZE = 4
#     NUM_EPOCHS = 50
#     LEARNING_RATE = 1e-4
#     IMG_SIZE = 512
    
#     # Device configuration for Apple Silicon (M1/M2/M3) or CUDA
#     if torch.backends.mps.is_available():
#         DEVICE = torch.device('mps')
#         print("Using Apple Silicon GPU (MPS)")
#     elif torch.cuda.is_available():
#         DEVICE = torch.device('cuda')
#         print("Using CUDA GPU")
#     else:
#         DEVICE = torch.device('cpu')
#         print("Using CPU")
    
#     print(f"Device: {DEVICE}")
    
#     # Load data
#     print("Loading data...")
#     train_df = pd.read_csv(TRAIN_CSV)
#     print(f"Training samples: {len(train_df)}")
    
#     # Split train/val
#     from sklearn.model_selection import train_test_split
#     train_df, val_df = train_test_split(train_df, test_size=0.15, random_state=42)
    
#     # Data augmentation
#     train_transform = A.Compose([
#         A.Resize(IMG_SIZE, IMG_SIZE),
#         A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
#         A.RandomRotate90(p=0.5),
#         A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p=0.5),
#         A.RandomBrightnessContrast(p=0.3),
#         A.HueSaturationValue(p=0.3),
#         A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#         ToTensorV2()
#     ])
    
#     val_transform = A.Compose([
#         A.Resize(IMG_SIZE, IMG_SIZE),
#         A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#         ToTensorV2()
#     ])
    
#     # Create datasets
#     train_dataset = NucleiDataset(train_df, TRAIN_DIR, transform=train_transform, is_train=True)
#     val_dataset = NucleiDataset(val_df, TRAIN_DIR, transform=val_transform, is_train=True)
    
#     train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
#     val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
#     # Create model
#     print("Creating model...")
#     model = get_model(num_classes=5, encoder='resnet50', encoder_weights='imagenet')
#     model = model.to(DEVICE)
    
#     # Loss and optimizer
#     criterion = CombinedLoss(num_classes=5)
#     optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
#     scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
    
#     # Training loop
#     print("Starting training...")
#     best_val_loss = float('inf')
    
#     for epoch in range(NUM_EPOCHS):
#         print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
        
#         train_loss = train_epoch(model, train_loader, criterion, optimizer, DEVICE)
#         val_loss = validate_epoch(model, val_loader, criterion, DEVICE)
#         scheduler.step()
        
#         print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
        
#         # Save best model
#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             torch.save(model.state_dict(), 'best_model.pth')
#             print(f"Model saved! Best Val Loss: {best_val_loss:.4f}")
    
#     # Load best model for inference
#     print("\nLoading best model for inference...")
#     model.load_state_dict(torch.load('best_model.pth'))
    
#     # Generate predictions
#     print("Generating predictions...")
#     test_images = sorted(TEST_DIR.glob('*.tif'))
    
#     submission = []
    
#     for img_path in tqdm(test_images, desc='Predicting'):
#         img_id = img_path.stem
        
#         # Load and preprocess image
#         image = cv2.imread(str(img_path))
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         orig_h, orig_w = image.shape[:2]
        
#         # Transform
#         transformed = val_transform(image=image)
#         image_tensor = transformed['image']
        
#         # Predict instances
#         instance_masks = predict_instances(model, image_tensor, DEVICE)
        
#         # Resize back to original size
#         row_data = {'image_id': img_id}
#         for class_name in ['Epithelial', 'Lymphocyte', 'Neutrophil', 'Macrophage']:
#             mask = instance_masks[class_name]
#             mask_resized = cv2.resize(mask, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)
#             rle = rle_encode_instance_mask(mask_resized)
#             row_data[class_name] = rle
        
#         submission.append(row_data)
    
#     # Create submission file
#     submission_df = pd.DataFrame(submission)
#     submission_df = submission_df[['image_id', 'Epithelial', 'Lymphocyte', 'Neutrophil', 'Macrophage']]
#     submission_df.to_csv('submission.csv', index=False)
#     print("\nSubmission saved to submission.csv")


# if __name__ == '__main__':
#     main()

# ==================== Main Pipeline (Modified for Multi-Model Training/Inference) ====================

def main_multi_model():
    # ... (Configuration, Device, and Data loading remains the same) ...
    # ... (Train/Val split and Data augmentation remains the same) ...
    
    # Configuration
    DATA_DIR = Path('kaggle-data')
    TRAIN_DIR = DATA_DIR / 'train'
    TEST_DIR = DATA_DIR / 'test_final'
    TRAIN_CSV = DATA_DIR / 'train_ground_truth.csv'
    
    BATCH_SIZE = 4
    NUM_EPOCHS = 50
    LEARNING_RATE = 1e-4
    IMG_SIZE = 512
    
    # Device configuration
    if torch.backends.mps.is_available():
        DEVICE = torch.device('mps')
        print("Using Apple Silicon GPU (MPS)")
    # ... (rest of device configuration) ...
    elif torch.cuda.is_available():
        DEVICE = torch.device('cuda')
        print("Using CUDA GPU")
    else:
        DEVICE = torch.device('cpu')
        print("Using CPU")
    
    print(f"Device: {DEVICE}")
    
    # Load data
    print("Loading data...")
    train_df = pd.read_csv(TRAIN_CSV)
    print(f"Training samples: {len(train_df)}")
    
    # Split train/val
    from sklearn.model_selection import train_test_split
    train_df, val_df = train_test_split(train_df, test_size=0.15, random_state=42)
    
    # Data augmentation (same as original)
    train_transform = A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.HueSaturationValue(p=0.3),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
    
    val_transform = A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

    
    CLASSES = ['Epithelial', 'Lymphocyte', 'Neutrophil', 'Macrophage']
    models = {}

    # --- Training Loop for each Class ---
    print("\nStarting multi-model training...")
    
    for class_name in CLASSES:
        print(f"\n================ Training Model for: {class_name} ================")
        
        # Create datasets specific to this class
        train_dataset = NucleiDataset(train_df, TRAIN_DIR, transform=train_transform, is_train=True, target_class=class_name)
        val_dataset = NucleiDataset(val_df, TRAIN_DIR, transform=val_transform, is_train=True, target_class=class_name)
        
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        
        # Create model (num_classes=2 for binary)
        model = get_model(num_classes=2, encoder='resnet50', encoder_weights='imagenet').to(DEVICE)
        
        # Loss and optimizer
        criterion = CombinedBinaryLoss(device=DEVICE)
        optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
        
        best_val_loss = float('inf')
        model_save_path = f'best_model_{class_name}.pth'
        
        for epoch in range(NUM_EPOCHS):
            print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
            
            # Use the original train/validate functions (they are generic for any model/loss)
            train_loss = train_epoch(model, train_loader, criterion, optimizer, DEVICE)
            val_loss = validate_epoch(model, val_loader, criterion, DEVICE)
            scheduler.step()
            
            print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
            
            # Save best model
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), model_save_path)
                print(f"Model saved! Best Val Loss: {best_val_loss:.4f}")
        
        # Load best weights for final use
        model.load_state_dict(torch.load(model_save_path))
        models[class_name] = model
    
    # --- Inference ---
    print("\nLoading best models for inference and generating predictions...")
    test_images = sorted(TEST_DIR.glob('*.tif'))
    submission = []
    
    for img_path in tqdm(test_images, desc='Predicting'):
        img_id = img_path.stem
        
        # Load and preprocess image (same as original)
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        orig_h, orig_w = image.shape[:2]
        
        # Transform
        transformed = val_transform(image=image)
        image_tensor = transformed['image']
        
        # Predict instances using the collection of models
        instance_masks = predict_instances(models, image_tensor, DEVICE)
        
        # Resize back to original size and RLE encode (same as original)
        row_data = {'image_id': img_id}
        for class_name in CLASSES:
            mask = instance_masks[class_name]
            mask_resized = cv2.resize(mask, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)
            rle = rle_encode_instance_mask(mask_resized)
            row_data[class_name] = rle
        
        submission.append(row_data)
    
    # Create submission file (same as original)
    submission_df = pd.DataFrame(submission)
    submission_df = submission_df[['image_id'] + CLASSES]
    submission_df.to_csv('submission.csv', index=False)
    print("\nSubmission saved to submission.csv")


if __name__ == '__main__':
    main_multi_model()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using Apple Silicon GPU (MPS)
Device: mps
Loading data...
Training samples: 209

Starting multi-model training...

================ Training Model for: Epithelial ================
Epoch 1/50


Validation: 100%|██████████| 8/8 [00:05<00:00,  1.58it/s]


Train Loss: 0.8886 | Val Loss: 0.9464
Model saved! Best Val Loss: 0.9464
Epoch 2/50


Validation: 100%|██████████| 8/8 [00:04<00:00,  1.68it/s]


...

Train Loss: 0.7027 | Val Loss: 0.8535
Epoch 50/50


Validation: 100%|██████████| 8/8 [00:05<00:00,  1.60it/s]


Train Loss: 0.6895 | Val Loss: 0.8359

Loading best models for inference and generating predictions...


Predicting: 100%|██████████| 40/40 [00:25<00:00,  1.57it/s]



Submission saved to submission.csv
